In [1]:
import dimod
import neal
import numpy as np
from dwave.system import DWaveSampler, EmbeddingComposite

# Solve Natural Linear Program as QUBO with default Solver Advantage_system1.1 

Problem ID: ec695b47-3af8-4f5f-bfe4-b58fe3185fcb

In [2]:
A_int = np.array([[0,1,2,3,4,1],
              [2,1,4,1,0,2],
              [1,5,2,3,1,1]])
m, n = A_int.shape
vartype = dimod.Vartype.BINARY
c_int = np.array([1,2,1,-3,1,-1])
b = np.array([7,8,5])
print(A_int.dot(np.array([1,0,0,0,1,3])))

[7 8 5]


# Problem statement:
Given $A\in Z^{mxn}, b\in Z^{m}, c\in Z^{n}$ 

find $x\in N^n$ with:

$\min c^T x $ with constraint $Ax=b$ 



This is equivalent to solving QUBO:

$min_x c^Tx + x^T A^T A x - 2 b^TAx + b^Tb$

eventually we need to set weights on the penalty parts $x^T A^T A x - 2 b^TAx + b^Tb$

In [3]:
def get_binary_form(c,bits):
    c_bin = []
    for c_j in c:
        c_bin += [c_j * 2**i for i in range(0,bits)]
    return np.array(c_bin)

In [4]:
# represent each x_i as 3 bits to the power of 2
bits = 3
# Set Penalty for Constraints aprox solution
P = max(abs(c_int))
c = get_binary_form(c_int,bits)
print(c)
A = np.array([get_binary_form(A_int[i], bits) for i in range(0,m)])
print(A)

[  1   2   4   2   4   8   1   2   4  -3  -6 -12   1   2   4  -1  -2  -4]
[[ 0  0  0  1  2  4  2  4  8  3  6 12  4  8 16  1  2  4]
 [ 2  4  8  1  2  4  4  8 16  1  2  4  0  0  0  2  4  8]
 [ 1  2  4  5 10 20  2  4  8  3  6 12  1  2  4  1  2  4]]


In [5]:
# Set up linear and quadratic coefficients
L = c.T - P * 2 * np.matmul(b.T,A)
Q = P * np.matmul(A.T,A)
offset = P * b.T.dot(b)
bqm = dimod.as_bqm(L, Q, offset, vartype)

In [6]:
exactSolver = dimod.ExactSolver()
sim = neal.SimulatedAnnealingSampler()
num_reads = 1000
exactSet = exactSolver.sample(bqm)
simSet = sim.sample(bqm, num_reads = num_reads)

In [7]:
def construct_x(sample, bits):
    x=[]
    for k in range(0,len(sample.keys()),bits):
        x += [sum(sample[k+j]* 2**j for j in range(0,bits))]
    return np.array(x)

In [8]:
def testSamples(sampleset, bits):
    print("Minimum Energy Level found in:",sampleset.first)
    X = []
    for sample, energy,  in sampleset.data(fields=['sample', 'energy']):
        x = construct_x(sample,bits)
        if all(A_int.dot(x)[i] == b[i] for i in range(0,m)) :
            X += [(x,energy)]
    return X

In [9]:
solutions_exact = testSamples(exactSet, bits)
for x, energy in solutions_exact :
    print("Found solution {} with Energy Level {}".format(x,energy))
        

Minimum Energy Level found in: Sample(sample={0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 1, 16: 1, 17: 0}, energy=-1.0, num_occurrences=1)
Found solution [1 0 0 0 1 3] with Energy Level -1.0
Found solution [1 0 1 0 1 1] with Energy Level 2.0


In [10]:
solutions_neal = testSamples(simSet, bits)
solutions_unique = { str(x) : energy for x, energy in solutions_neal}
solutions_count = {x:0 for x in solutions_unique.keys()}
for x,energy in solutions_neal:
    solutions_count[str(x)]+=1
for x in solutions_unique.keys() :
    print("Found solution {} {} times with Energy Level {}".format(x,solutions_count[x],solutions_unique[x]))

Minimum Energy Level found in: Sample(sample={0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 1, 16: 1, 17: 0}, energy=-1.0, num_occurrences=1)
Found solution [1 0 0 0 1 3] 218 times with Energy Level -1.0
Found solution [1 0 1 0 1 1] 114 times with Energy Level 2.0


In [11]:
sampler = EmbeddingComposite(DWaveSampler())
sampleset = sampler.sample(bqm,num_reads = num_reads)

In [12]:
sampleset.to_pandas_dataframe()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,chain_break_fraction,energy,num_occurrences
0,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,0,0.000000,-1.0,3
1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0.000000,0.0,1
2,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,1,0,0.000000,3.0,1
3,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0.000000,4.0,1
4,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0.000000,6.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731,0,0,0,1,0,0,1,0,1,0,...,0,1,1,0,0,0,0,0.000000,1792.0,1
732,1,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0.055556,127.0,1
733,0,0,0,1,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0.000000,1922.0,1
734,1,0,1,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0.000000,2322.0,1


In [13]:
solutions_real = testSamples(sampleset, bits)
solutions_unique = { str(x) : energy for x, energy in solutions_real}
for x in solutions_unique.keys() :
    print("Found solution {} with Energy Level {}".format(x,solutions_unique[x]))

Minimum Energy Level found in: Sample(sample={0: 1, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 1, 13: 0, 14: 0, 15: 1, 16: 1, 17: 0}, energy=-1.0, num_occurrences=3, chain_break_fraction=0.0)
Found solution [1 0 0 0 1 3] with Energy Level -1.0
